<h3>Introduction</h3>

<p>In this project we are making api calls to two different apis. We chose a location
and collected json formated information about our points of interest from these two apis.
After creating a sql data base, we compared the two apis. The comparison is based on the number of
certain point of interest</p>

In [613]:
#import necessary libraries
import pandas as pd
import numpy as np
import requests as re
import json
from IPython.display import JSON 
import os
import sqlite3 as sqlite
from sqlalchemy import create_engine
import requests_cache
from pandas import json_normalize

In [614]:
#save the data into the cache
requests_cache.install_cache()
#create connection to sql
engine = create_engine('sqlite://',echo=False)

In [723]:
#FOR YELP:
#NOTE: We could set environmental variables here and use
#os.environ to grab the key

api_key_yelp='6jzXY72Qz52SlqSsOQ1oPpnp5VD4HaMLkWKOH40PisqqynCz3O2_UfgpyQkMmapcAmGp8W1xFevpkWJgY99Gk0xhS7G2DLbCusHxpIfOiRpsyY2YkUSuqdl10rFsX3Yx'
headers_yelp = {'Authorization': 'Bearer %s' % api_key_yelp}
url_yelp='https://api.yelp.com/v3/businesses/search'

In [616]:
#Function to make query for bar and gym
#First there were distinct functions to make
#api calls for each POI.

def bar_four(locstring):
    
    url = 'https://api.foursquare.com/v2/venues/search'
    params = dict(
    client_id='QSPKXLAWDU4ULH1G044BFW4ETF31K0PQ4RYUV5VV3W1YYIOW',
    client_secret='T0DSRMUKYZXB4NLODWYUITVSQYGMXKRPQXG0J04B3CXKREGR',
    v='20180323',
    ll=locstring,
    query='bar,gym',
    radius = 500,
    limit=20
    )
    
    try:
        resp = re.get(url=url, params=params)
        resp_json = json.loads(resp.text)
        
    except Exception as e:
        print(e)
    return resp_json['response']['venues']


In [725]:
#Function for making request to the YELP Api

def query_yelp(loc_tuple,url_yelp,headers_yelp,typeof):
    params = {'term':typeof, 'latitude':loc_tuple[0] ,'longitude':loc_tuple[1], 'radius':'500', 'limit':'30'}
    try:
        req = re.get(url = url, params = params, headers = headers_yelp)
    except Exception as e:
        print(e)
    req_json = json.loads(req.text)
    return req_json['bussinesses']

In [653]:
#This step is redundant, making random locations were chosen
#and they were saved in a txt file name 'locations.txt'
#make a list of those locations

list_loc = []
with open('locations.txt') as f:
    for i in range(0,50):
        list_loc.append(f.readline())
list_loc=[x.strip() for x in list_loc]

In [654]:
#this step is also not needed. just choosing the 
#first location from the list of locations to make api call

loc_sample_four = list_loc[0]

#for the yelp api it is slightly different.
#foursquare accepts ll as one string, for yelp
#i had to make a tuple and call the function

loc_yelp = []
for value in list_loc:
    x,y = value.split(',')
    loc_yelp.append((x,y))
    
#Location for Yelp; passing a tuple to the function

loc_sample_yelp = loc_yelp[0]


<h3>Foursquare</h3>

In [656]:
#foursquare request for bars
json_req_bar = bar_four(loc_sample_four)

In [657]:
JSON(json_req_bar)

<IPython.core.display.JSON object>

In [658]:
#saving json response to a json file for later use
#this step is extra could make a normalized json file
#without going through this step

with open('foursquare.json','w',encoding='utf-8') as f:
    json.dump(json_req_bar, f)

In [659]:
#getting the content of the json file to normalize it
with open('foursquare.json') as s:
    nested_json_bar = json.load(s)
print(type(nested_json_bar))

<class 'list'>


In [660]:
#Normalize the jason file
foursquare_normalized = json_normalize(nested_json_bar)

In [661]:
#list the categories in the lcoation
#categories contained important informatio like
#id for each location

foursquare_normalized_categories = json_normalize(nested_json, record_path = 'categories')

#get the name of category and id:
four_category_table = foursquare_normalized_categories[['id','name']]

In [662]:
#extra: you can merge two tables to get the location for each category
#note: for the foursquare range is set to 5km thats why we have
#several lat,long

In [735]:
foursquare_normalized_categories

,id,name,pluralName,shortName,primary,icon.prefix,icon.suffix
0,4bf58dd8d48988d116941735,Bar,Bars,Bar,True,https://ss3.4sqi.net/img/categories_v2/nightli...,.png
1,4bf58dd8d48988d116941735,Bar,Bars,Bar,True,https://ss3.4sqi.net/img/categories_v2/nightli...,.png
2,4bf58dd8d48988d116941735,Bar,Bars,Bar,True,https://ss3.4sqi.net/img/categories_v2/nightli...,.png
3,4bf58dd8d48988d116941735,Bar,Bars,Bar,True,https://ss3.4sqi.net/img/categories_v2/nightli...,.png
4,4bf58dd8d48988d1e5931735,Music Venue,Music Venues,Music Venue,True,https://ss3.4sqi.net/img/categories_v2/arts_en...,.png
5,4bf58dd8d48988d11d941735,Sports Bar,Sports Bars,Sports Bar,True,https://ss3.4sqi.net/img/categories_v2/nightli...,.png
6,4bf58dd8d48988d14e941735,American Restaurant,American Restaurants,American,True,https://ss3.4sqi.net/img/categories_v2/food/de...,.png
7,4bf58dd8d48988d145941735,Chinese Restaurant,Chinese Restaurants,Chinese,True,https://ss3.4sqi.net/img/categories_v2/food/as...,.png
8,4bf58dd8d48988d14b941735,Winery,Wineries,Winery,True,https://ss3.4sqi.net/img/categories_v2/food/wi...,.png
9,4bf58dd8d48988d1e0931735,Coffee Shop,Coffee Shops,Coffee Shop,True,https://ss3.4sqi.net/img/categories_v2/food/co...,.png


In [736]:
#This is also an extra step. we could make use of
#the above table to make our calculation, but its nice
#to have the locations of the bar as well.

four_loct = foursquare_normalized[['location.lat','location.lng']]
loc_cat_merged = pd.merge(four_loct, four_category_table, right_index=True, left_index=True)
loc_cat_merged

,location.lat,location.lng,id,name
0,39.165297,-86.534881,4bf58dd8d48988d116941735,Bar
1,39.162231,-86.535461,4bf58dd8d48988d116941735,Bar
2,39.165387,-86.534669,4bf58dd8d48988d116941735,Bar
3,39.157078,-86.535104,4bf58dd8d48988d116941735,Bar
4,39.163398,-86.534609,4bf58dd8d48988d1e5931735,Music Venue
5,39.161457,-86.539153,4bf58dd8d48988d11d941735,Sports Bar


In [737]:
four_category_table

,id,name
0,4bf58dd8d48988d116941735,Bar
1,4bf58dd8d48988d116941735,Bar
2,4bf58dd8d48988d116941735,Bar
3,4bf58dd8d48988d116941735,Bar
4,4bf58dd8d48988d1e5931735,Music Venue
5,4bf58dd8d48988d11d941735,Sports Bar
6,4bf58dd8d48988d14e941735,American Restaurant
7,4bf58dd8d48988d145941735,Chinese Restaurant
8,4bf58dd8d48988d14b941735,Winery
9,4bf58dd8d48988d1e0931735,Coffee Shop


In [665]:
#creating sql database out of the tables for Foursquare

four_category_table.to_sql('foursquare', con=engine,if_exists='replace')
loc_cat_merged.to_sql('foursquare_extra', con=engine,if_exists='replace')

In [734]:
#to ficth the data just comment out:

#engine.execute("SELECT * FROM foursquare_extra").fetchall()
#engine.execute("SELECT * FROM foursquare_extra").fetchall()

[(0, 39.165297, -86.5348814, '4bf58dd8d48988d116941735', 'Bar'),
 (1, 39.1622314453125, -86.53546142578125, '4bf58dd8d48988d116941735', 'Bar'),
 (2, 39.16538683322792, -86.53466905898989, '4bf58dd8d48988d116941735', 'Bar'),
 (3, 39.157078000000006, -86.535104, '4bf58dd8d48988d116941735', 'Bar'),
 (4, 39.163398, -86.534609, '4bf58dd8d48988d1e5931735', 'Music Venue'),
 (5, 39.16145691211021, -86.53915263050779, '4bf58dd8d48988d11d941735', 'Sports Bar')]

<h2>Yelp</h2>

In [732]:
#Making the request:
json_req_yelp = query_yelp(loc_sample_yelp,url,headers_yelp,'bar,gym')

In [733]:
JSON(json_req_yelp)

<IPython.core.display.JSON object>

In [730]:
normalize_yelp = json_normalize(json_req_yelp, record_path = 'categories',meta = ['name','id'])

KeyError: 'categories'

In [731]:
#Yelp gives different alias for each location!
normalize_yelp_right_order = normalize_yelp[['name','id','alias','title']]

In [672]:
normalize_yelp_right_order

,name,id,alias,title
0,Hyatt Place Bloomington,q12cFw0byXb0KjCXIHetKQ,hotels,Hotels
1,Hyatt Place Bloomington,q12cFw0byXb0KjCXIHetKQ,venues,Venues & Event Spaces
2,Hyatt Place Bloomington,q12cFw0byXb0KjCXIHetKQ,cocktailbars,Cocktail Bars
3,Turkuaz Cafe,uOdmDKWdSsd-qc0HcrNyzQ,mediterranean,Mediterranean
4,Turkuaz Cafe,uOdmDKWdSsd-qc0HcrNyzQ,turkish,Turkish
5,Turkuaz Cafe,uOdmDKWdSsd-qc0HcrNyzQ,wine_bars,Wine Bars


In [673]:
#Making a sql databse 
normalize_yelp_right_order.to_sql('yelp',con=engine, if_exists='replace')

In [674]:
#to see that it really exists; comment out below:
#engine.execute("SELECT * FROM yelp").fetchall()

<h2>Comparing the data</h2>

In [675]:
#filtering data based on gym and bars for yelp
yelp_gym = normalize_yelp_right_order.loc[normalize_yelp_right_order['alias'] == 'gyms']
yelp_bars = normalize_yelp_right_order.loc[(normalize_yelp['alias'] == 'bars') | (normalize_yelp['alias'] == 'wine_bars')]

In [676]:
yelp_bars.value_counts()

name          id                      alias      title    
Turkuaz Cafe  uOdmDKWdSsd-qc0HcrNyzQ  wine_bars  Wine Bars    1
dtype: int64

In [677]:
yelp_gym.value_counts()

Series([], dtype: int64)

In [678]:
#you could filter the data based on gym and bars for foursquare
#and count the numbers like Yelp
#loc_cat_merged_bar = loc_cat_merged.loc[loc_cat_merged['id'] == '4bf58dd8d48988d116941735']

In [679]:
#or we could make a table that shows number of each category
#and compare it
loc_cat_merged.groupby(['id','name']).size().unstack()

name,Bar,Music Venue,Sports Bar
id,,,
4bf58dd8d48988d116941735,4.0,NaN,NaN
4bf58dd8d48988d11d941735,NaN,NaN,1.0
4bf58dd8d48988d1e5931735,NaN,1.0,NaN


In [738]:
four_category_table.groupby(['id','name']).size().unstack()

name,American Restaurant,Bar,Bookstore,Chinese Restaurant,Cocktail Bar,Coffee Shop,College Lab,Food,Fraternity House,Gym,...,Music Venue,Night Market,Residential Building (Apartment / Condo),Salon / Barbershop,Snack Place,Soccer Field,Sports Bar,Sushi Restaurant,Tex-Mex Restaurant,Winery
id,,,,,,,,,,,,,,,,,,,,,
4bf58dd8d48988d110951735,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN
4bf58dd8d48988d114951735,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4bf58dd8d48988d116941735,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4bf58dd8d48988d11d941735,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
4bf58dd8d48988d11e941735,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4bf58dd8d48988d145941735,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4bf58dd8d48988d14b941735,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4bf58dd8d48988d14e941735,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4bf58dd8d48988d176941735,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<h3>Choose top 10 POIS</h3>

In [721]:
url_3 = 'https://api.foursquare.com/v2/venues/4bf58dd8d48988d116941735'
params = {
    'client_id':'QSPKXLAWDU4ULH1G044BFW4ETF31K0PQ4RYUV5VV3W1YYIOW',
    'client_secret':'T0DSRMUKYZXB4NLODWYUITVSQYGMXKRPQXG0J04B3CXKREGR',
    'v':'20180323'
}
rating_req = re.get(url,params)
rating_json = json.loads(rating_req.text)

In [707]:
rating_json

{'meta': {'code': 400,
  'errorType': 'param_error',
  'errorDetail': 'Value 4bf58dd8d48988d116941735 is invalid for venue id',
  'requestId': '5f6d4b351c56f460ddf598ea'},
 'response': {}}

,alias,title,name,id
0,hotels,Hotels,Hyatt Place Bloomington,q12cFw0byXb0KjCXIHetKQ
1,venues,Venues & Event Spaces,Hyatt Place Bloomington,q12cFw0byXb0KjCXIHetKQ
2,cocktailbars,Cocktail Bars,Hyatt Place Bloomington,q12cFw0byXb0KjCXIHetKQ
3,mediterranean,Mediterranean,Turkuaz Cafe,uOdmDKWdSsd-qc0HcrNyzQ
4,turkish,Turkish,Turkuaz Cafe,uOdmDKWdSsd-qc0HcrNyzQ
5,wine_bars,Wine Bars,Turkuaz Cafe,uOdmDKWdSsd-qc0HcrNyzQ


<h3>Conlusion</h3>